In [2]:
!pip install pandas 
!pip install psycopg2-binary
!pip install sqlalchemy


[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
  Using cached SQLAlchemy-2.0.31-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)

[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install kafka-python

  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)

[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret():

    secret_name = "DBCreds"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    # Parse the secret string to get the credentials
    secret_dict = json.loads(secret)
    username = secret_dict['username']
    password = secret_dict['password']
    host = secret_dict['host']
    port = secret_dict['port']
    dbname = secret_dict['dbname']

    return username, password, host, port, dbname


(user,pswd,host,port,db) = get_secret()

In [ ]:
import json
from kafka import KafkaConsumer
import psycopg2
from psycopg2.extras import execute_values

# PostgreSQL database connection details
db_config = {
    'dbname': db,
    'user': user,
    'password': pswd,
    'host': host,
    'port': port
}

# Kafka topic
postgres_topic = "postgresql"
brokers = [f"{host}:9092"]

# Create a Kafka consumer
consumer = KafkaConsumer(
    postgres_topic,
    bootstrap_servers=brokers,
    value_deserializer=lambda message: json.loads(message.decode('utf-8'))
)

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    print("Connected to PostgreSQL successfully.")
except Exception as e:
    print(f"Failed to connect to PostgreSQL: {e}")
    exit()

try:
    print("Starting to consume messages.")
    for message in consumer:
        data = message.value
        print(f"Received message: {data}")
        uid = data.pop('uid')
        data.pop('legitimate')
        try:
            insert_query = """
            INSERT INTO malware_data (uid, features)
            VALUES (%s, %s)
            """
            cursor.execute(insert_query, (uid, json.dumps(data)))
            conn.commit()
            print("Data inserted successfully.")
        except Exception as e:
            print(f"Failed to insert data: {e}")
            conn.rollback()
        
        
except KeyboardInterrupt:
    print("Consumer interrupted.")
finally:
    # Close PostgreSQL connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()
        print("PostgreSQL connection closed.")


Connected to PostgreSQL successfully.
Starting to consume messages.
Received message: {'name': 'memtest.exe', 'md5': '631ea355665f28d4707448e442fbf5b8', 'machine': 332, 'sizeofoptionalheader': 224, 'characteristics': 258, 'majorlinkerversion': 9, 'minorlinkerversion': 0, 'sizeofcode': 361984, 'sizeofinitializeddata': 115712, 'sizeofuninitializeddata': 0, 'addressofentrypoint': 6135, 'baseofcode': 4096, 'baseofdata': 372736, 'imagebase': 4194304.0, 'sectionalignment': 4096, 'filealignment': 512, 'majoroperatingsystemversion': 0, 'minoroperatingsystemversion': 0, 'majorimageversion': 0, 'minorimageversion': 0, 'majorsubsystemversion': 1, 'minorsubsystemversion': 0, 'sizeofimage': 1036288, 'sizeofheaders': 1024, 'checksum': 485887, 'subsystem': 16, 'dllcharacteristics': 1024, 'sizeofstackreserve': 1048576, 'sizeofstackcommit': 4096, 'sizeofheapreserve': 1048576, 'sizeofheapcommit': 4096, 'loaderflags': 0, 'numberofrvaandsizes': 16, 'sectionsnb': 8, 'sectionsmeanentropy': 5.7668065537, 'se